# Wildfire

In [134]:
import sys

# required 3.12 version
if [sys.version_info[i] for i in range(3)][:2] != [3, 12]: 
    raise Exception(f"Python 3.12 is required (Current is {[sys.version_info[i] for i in range(3)]})")

# import os
# import time
# import random
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import cv2
# import imutils
# import numpy as np
# from PIL import Image
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader

import os
from pathlib import Path
import pandas
from pandas import DataFrame

In [135]:
import torch

PROGRAM_NAME: str = "Wildfire"
CATEGORIES: list[str] = ["fire", "nofire"]
DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

In [136]:
def short_path_name(path: Path) -> str:
    parts = path.parts
    return '.../' + '/'.join(parts[-4:]) if len(parts) > 4 else str(path)

def append_path_or_else(base: Path, paths: list[str]) -> Path:
    for path in paths:
        maybe_path = base / path
        if maybe_path.is_dir():
            return maybe_path
    return base

In [137]:
def generate_dataframe(folder_path: Path) -> DataFrame:
    print(f"[{PROGRAM_NAME}]: Generating new dataframe from '{short_path_name(folder_path)}'", end=" ")
    dataframe: DataFrame = DataFrame([], columns = ["Image", "Class"])
    
    FIRE_DIR: Path = append_path_or_else(folder_path, ["fire", "wildfire"])
    NOFIRE_DIR: Path = append_path_or_else(folder_path, ["nofire", "nowildfire"])
    
    fire_images: list[tuple[str, str]] = [(img.name, "fire") for img in FIRE_DIR.iterdir() if img.is_file()]
    nofire_images: list[tuple[str, str]] = [(img.name, "nofire") for img in NOFIRE_DIR.iterdir() if img.is_file()]

    data: list[tuple[str, str]] = fire_images + nofire_images
    dataframe: DataFrame = DataFrame(data, columns=["Image", "Class"])
    
    print(dataframe.shape)

    return dataframe

Dataset 1: `jafar_2023`
> Ibn Jafar, Anam; Islam, Al Mohimanul ; Binta Masud, Fatiha; Ullah, Jeath Rahmat; Ahmed, Md. Rayhan (2023), “FlameVision : A new dataset for wildfire classification and detection using aerial imagery ”, Mendeley Data, V4. https://doi.org/10.17632/fgvscdjsmt.4

Dataset 2: `madafri_2023`
> El-Madafri I, Peña M, Olmedo-Torre N. The Wildfire Dataset: Enhancing Deep Learning-Based Forest Fire Detection with a Diverse Evolving Open-Source Dataset Focused on Data Representativeness and a Novel Multi-Task Learning Approach. Forests. 2023; 14(9):1697. https://doi.org/10.3390/f14091697

Dataset 3: `aaba_2022`
> Aaba, A. (2022). Wildfire Prediction Dataset (Satellite Images) [Data set]. Kaggle. https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset/data

In [138]:
#################################################################
#                                                               #
#  When you download all datasets:                              #
#    1. Renamed parent folders `lastname_year`.                 #
#    1. Renamed child folders `fire` or `nofire` respectively.  #
#    2. Keep the file structure unmodified.                     #
#                                                               #
#################################################################

'''
Wildfire
├───wildfire.ipynb
└───data
    ├───aaba_2022
    │   ├───test
    │   │   ├───nowildfire
    │   │   └───wildfire
    │   ├───train
    │   │   ├───nowildfire
    │   │   └───wildfire
    │   └───valid
    │       ├───nowildfire
    │       └───wildfire
    ├───jafar_2023
    │   ├───Classification
    │   │   ├───test
    │   │   │   ├───fire
    │   │   │   └───nofire
    │   │   ├───train
    │   │   │   ├───fire
    │   │   │   └───nofire
    │   │   └───valid
    │   │       ├───fire
    │   │       └───nofire
    │   └───Detection
    │       ├───test
    │       │   ├───annotations
    │       │   └───images
    │       ├───train
    │       │   ├───annotations
    │       │   └───images
    │       └───valid
    │           ├───annotations
    │           └───images
    └───madafri_2023
        ├───test
        │   ├───fire
        │   └───nofire
        ├───train
        │   ├───fire
        │   └───nofire
        └───val
            ├───fire
            └───nofire
'''

__file__ = os.getcwd()
BASE_DIR = Path(__file__).resolve().parent / PROGRAM_NAME
DATA_DIR: Path = BASE_DIR / "data"

class WildFireData1():
    def __init__(self) -> None:
        self.DIR: Path = DATA_DIR / "jafar_2023"
        self.CLASS_DIR: Path = self.DIR / "Classification"
        self.CLASS_TRAIN_DIR: Path = self.CLASS_DIR / "train"
        self.CLASS_TEST_DIR: Path = self.CLASS_DIR / "test"
        self.CLASS_VALID_DIR: Path = self.CLASS_DIR / "valid"
        self.DETECTION_DIR: Path = self.DIR / "Detection"
        self.DETECTION_TRAIN_DIR: Path = self.DETECTION_DIR / "train"
        self.DETECTION_TEST_DIR: Path = self.DETECTION_DIR / "test"
        self.DETECTION_VALID_DIR: Path = self.DETECTION_DIR / "valid"
    
    def generate_dataframes(self) -> list[DataFrame]:
        return [
            generate_dataframe(self.CLASS_TRAIN_DIR),
            generate_dataframe(self.CLASS_TEST_DIR),
            generate_dataframe(self.CLASS_VALID_DIR),
        ]

class WildFireData2():
    def __init__(self) -> None:
        self.DIR: Path = DATA_DIR / "madafri_2023"
        self.TRAIN_DIR: Path = self.DIR / "train"
        self.TEST_DIR: Path = self.DIR / "test"
        self.VALID_DIR: Path = self.DIR / "val"
    
    def generate_dataframes(self) -> list[DataFrame]:
        return [
            generate_dataframe(self.TRAIN_DIR),
            generate_dataframe(self.TEST_DIR),
            generate_dataframe(self.VALID_DIR),
        ]

class WildFireData3():
    def __init__(self) -> None:
        self.DIR: Path = DATA_DIR / "aaba_2022"
        self.TRAIN_DIR: Path = self.DIR / "train"
        self.TEST_DIR: Path = self.DIR / "test"
        self.VALID_DIR: Path = self.DIR / "valid"
    
    def generate_dataframes(self) -> list[DataFrame]:
        return [
            generate_dataframe(self.TRAIN_DIR),
            generate_dataframe(self.TEST_DIR),
            generate_dataframe(self.VALID_DIR),
        ]

In [ ]:
wildfire_data_1: WildFireData1 = WildFireData1()
wildfire_data_2: WildFireData2 = WildFireData2()
wildfire_data_3: WildFireData3 = WildFireData3()

dataframe_partial_1: list[DataFrame] = wildfire_data_1.generate_dataframes()
dataframe_partial_2: list[DataFrame] = wildfire_data_2.generate_dataframes()
dataframe_partial_3: list[DataFrame] = wildfire_data_3.generate_dataframes()

dataframe_partials: list[list[DataFrame]] = [
    dataframe_partial_1,
    dataframe_partial_2,
    dataframe_partial_3,
]

dataframe_train_partials: list[DataFrame] = [dataframe_partial[0] for dataframe_partial in dataframe_partials]
dataframe_test_partials: list[DataFrame] = [dataframe_partial[1] for dataframe_partial in dataframe_partials]
dataframe_valid_partials: list[DataFrame] = [dataframe_partial[2] for dataframe_partial in dataframe_partials]

[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/train' (6800, 2)
[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/test' (900, 2)
[Wildfire]: Generating new dataframe from '.../data/jafar_2023/Classification/valid' (900, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/train' (1887, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/test' (410, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/madafri_2023/val' (403, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/aaba_2022/train' (30250, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/aaba_2022/test' (6300, 2)
[Wildfire]: Generating new dataframe from '.../Wildfire/data/aaba_2022/valid' (6300, 2)


In [140]:
dataframe_train = pandas.concat(dataframe_train_partials, ignore_index=True)
dataframe_test = pandas.concat(dataframe_test_partials, ignore_index=True)
dataframe_valid = pandas.concat(dataframe_valid_partials, ignore_index=True)

print(dataframe_train)

                          Image   Class
0                  fire (1).png    fire
1                 fire (10).png    fire
2                fire (100).png    fire
3               fire (1000).png    fire
4               fire (1001).png    fire
...                         ...     ...
38932  -79.745321,43.752082.jpg  nofire
38933  -79.745322,43.688254.jpg  nofire
38934  -79.745551,43.736761.jpg  nofire
38935  -79.745792,43.791904.jpg  nofire
38936  -79.745993,43.715981.jpg  nofire

[38937 rows x 2 columns]
